In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
import seaborn as sns

from textblob import TextBlob
from transformers import pipeline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
csv = os.path.join(dirname,filename)

try:
    data = pd.read_csv(csv)
except Error:
    print("Error reading the csv")
finally:
    print(data)

In [ ]:
chat = ['nice hotel expensive parking got good deal stay hotel anniversary', 
        'arrived late evening took advice previous reviews did valet parking', 
        'check quick easy', 
        'little disappointed non-existent view room room clean nice size', 
        'bed comfortable woke stiff neck high pillows', 
        'not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway', 
        'maybe just noisy neighbors', 
        'aveda bath products nice', 
        'did not goldfish stay nice touch taken advantage staying longer', 
        'location great walking distance shopping', 
        'overall nice experience having pay 40 parking night']

In [ ]:
def moments(messages, window):
    ''' 
    Função que calcula a média de uma certa quantidade de polaridades que são definidas através de um valor inteiro positivo chamado de janela. 
    Cada média calculada dentro dessa janela é chamada de momento.
    '''
    f = lambda x: round(np.mean(x), 2)
    return [f(messages[i: i + window]) for i in range(len(messages) - window + 1)]

In [ ]:
def pad_moments(messages, window=3, first=True, length=5):
    """
    Função que calcula a janela dos últimos momentos de um chat.
    """
    lw = length + window - 1
    start, end = (0, lw) if (first) else (len(messages) - lw, len(messages))
    padded = messages[start:end]
    # m = moments(padded, window)
    return padded

In [ ]:
def get_padded_sessions(messages, window, first=False, length=5):
    """
    Função calcula a polaridade e faz o padding de todas as sessões presentes no dataframe.
    """
    padded_list = []
    for x in messages:
#        polarities = sentiment_analysis(df, session_id, method=sa_method)

        padded = [x, len(messages)]
        padded.extend(pad_moments(messages, window=window, first=False))
        if len(padded) == (length + 2):
            padded_list.append(padded)
    return padded_list

## TextBlob Sentiment Analysis

In [ ]:
def textblob_sentiment_analysis(msgs):
    ''' 
    Função que calcula as polaridades de uma lista de mensagens
    usando o TextBlob.
    '''
    polarities = []
    
    for i in range(len(msgs)):
        blob = TextBlob(msgs[i])
        if (len(blob.sentences) > 0):
            pol = blob.sentences[0].sentiment.polarity
            polarities.append(pol)
    
    return polarities

## Vader Sentiment Analyzer

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

"""
O vader retorna uma probabilidade da sentença ser negativa, neutra, positiva e um compound que quanto mais próximo de 1, 
mais positivo, e mais próximo de -1, mais negativo
"""
def vader_sentiment(sentences):
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        print("{:-<65} {}".format(sentence, str(vs)))

In [ ]:
padded_messages = get_padded_sessions(chat, 3)
padded_messages
# padded_messages = get_padded_sessions(chat, 3, False, 4)

In [ ]:
vader_sentiment(chat)